<a href="https://colab.research.google.com/github/KimJisanER/Dacon/blob/main/penguin_MassRegre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


path='/content/drive/MyDrive/penguin/'

train=pd.read_csv(path+'train.csv')
test=pd.read_csv(path+'test.csv')

print(train.head().T)
print(test.head().T)

                                                     0  ...                                  4
id                                                   0  ...                                  4
Species              Gentoo penguin (Pygoscelis papua)  ...  Gentoo penguin (Pygoscelis papua)
Island                                          Biscoe  ...                             Biscoe
Clutch Completion                                  Yes  ...                                 No
Culmen Length (mm)                                  50  ...                               49.6
Culmen Depth (mm)                                 15.3  ...                                 16
Flipper Length (mm)                                220  ...                                225
Sex                                               MALE  ...                               MALE
Delta 15 N (o/oo)                              8.30515  ...                            8.38324
Delta 13 C (o/oo)                             -25.

In [2]:
X_train=train.iloc[:,1:10]
y_train=train.loc[:,'Body Mass (g)']
test_id=test.iloc[:,0]
X_test=test.iloc[:,1:]


# print(X_train.T.head(11))
# print(y_train.T.head(11))
# print(test_id)

In [3]:
print(X_train.isnull().sum())
print(X_test.isnull().sum())

Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    3
Delta 15 N (o/oo)      3
Delta 13 C (o/oo)      3
dtype: int64
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    6
Delta 15 N (o/oo)      9
Delta 13 C (o/oo)      8
dtype: int64


In [4]:

pd.set_option('display.max_columns', 15)

print(X_train.describe())
print(X_test.describe())

       Culmen Length (mm)  Culmen Depth (mm)  Flipper Length (mm)  \
count          114.000000         114.000000           114.000000   
mean            44.613158          17.014912           203.052632   
std              5.321829           1.941363            14.653425   
min             33.500000          13.200000           174.000000   
25%             40.325000          15.225000           190.000000   
50%             45.200000          17.250000           199.000000   
75%             49.075000          18.600000           216.000000   
max             55.100000          21.100000           231.000000   

       Delta 15 N (o/oo)  Delta 13 C (o/oo)  
count         111.000000         111.000000  
mean            8.737634         -25.723051  
std             0.567698           0.859786  
min             7.632200         -27.018540  
25%             8.272585         -26.434025  
50%             8.632590         -25.955410  
75%             9.264635         -25.005945  
max       

In [5]:
X_train['Delta 15 N (o/oo)']=X_train['Delta 15 N (o/oo)'].fillna(X_train['Delta 15 N (o/oo)'].mean())
X_test['Delta 15 N (o/oo)']=X_test['Delta 15 N (o/oo)'].fillna(X_test['Delta 15 N (o/oo)'].mean())

X_train['Delta 13 C (o/oo)']=X_train['Delta 13 C (o/oo)'].fillna(X_train['Delta 13 C (o/oo)'].mean())
X_test['Delta 13 C (o/oo)']=X_test['Delta 13 C (o/oo)'].fillna(X_test['Delta 13 C (o/oo)'].mean())

print(X_train.isnull().sum())
print(X_test.isnull().sum())

Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    3
Delta 15 N (o/oo)      0
Delta 13 C (o/oo)      0
dtype: int64
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    6
Delta 15 N (o/oo)      0
Delta 13 C (o/oo)      0
dtype: int64


In [6]:
label=[]

for i in X_train.columns:
  if X_train.dtypes[i] == 'object':
    label.append(i)

print(label)

['Species', 'Island', 'Clutch Completion', 'Sex']


In [48]:
from sklearn.preprocessing import LabelEncoder
X_train.loc[:,label]=X_train.loc[:,label].apply(LabelEncoder().fit_transform)
X_test.loc[:,label]=X_test.loc[:,label].apply(LabelEncoder().fit_transform)

In [53]:
from sklearn.model_selection import train_test_split
nX_train,nX_val,ny_train,ny_val=train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [40]:
# from sklearn.preprocessing import StandardScaler
# scaler=StandardScaler()
# scaler.fit(nX_train)
# nX_train=scaler.fit_transform(nX_train)
# nX_val=scaler.fit_transform(nX_val)

In [70]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(max_depth=5, n_estimators=200, min_samples_leaf=8, min_samples_split=8)
model.fit(nX_train,ny_train)
print(model.score(nX_train,ny_train))
print(model.score(nX_val,ny_val))

0.8650746753558317
0.8270601635581216


In [68]:
param_grid={'max_depth':range(3,10),
            'n_estimators':range(100,400,100),
            'min_samples_leaf' : [8, 12, 18],
            'min_samples_split' : [8, 16, 20]}
from sklearn.model_selection import GridSearchCV
grid_search=GridSearchCV(RandomForestRegressor(),param_grid, cv=5)
grid_search.fit(nX_train,ny_train)

print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(nX_val, ny_val)))

Best Parameter: {'max_depth': 5, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 200}
Best Score: 0.8010
TestSet Score: 0.8329


In [71]:
import numpy as np

def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

In [72]:
pred=model.predict(nX_val)

RMSE(ny_val,pred)

285.8081528970343

In [73]:
from sklearn.ensemble import RandomForestRegressor
model_fin=RandomForestRegressor(max_depth=5, n_estimators=200, min_samples_leaf=8, min_samples_split=8)
model_fin.fit(X_train,y_train)
print(model_fin.score(nX_train,ny_train))

0.8597269098817826


In [82]:
pred_fin=model_fin.predict(X_test)
pred_fin=pd.DataFrame(pred_fin)

sub=pd.concat([test_id,pred_fin], axis=1)
sub.columns=['id','Body Mass (g)']
print(sub)

sub.to_csv('penguin_massregress.csv', index=False)
# print(pred_fin)


      id  Body Mass (g)
0      0    4398.207129
1      1    5495.344151
2      2    3478.370013
3      3    3443.880641
4      4    3463.524227
..   ...            ...
223  223    4330.343834
224  224    4830.656992
225  225    4617.971652
226  226    4010.245394
227  227    4027.344991

[228 rows x 2 columns]
